[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RelevanceAI/RelevanceAI-readme-docs/blob/v2.0.0/docs/getting-started/example-applications/_notebooks/RelevanceAI-ReadMe-Text-Search-using-USE-VectorHub.ipynb)

# Quickstart

to get features such as:
- hybrid search
- multivector search
- filtered search
- etc out of the box

# What I Need

- Project & API Key (grab your API key from https://cloud.relevance.ai/ in the settings area)
- Python 3
- Relevance AI Installed as shown below. You can also visit our [Installation guide](https://docs.relevance.ai/docs)

# Installation Requirements

In [ ]:
# remove `!` if running the line in a terminal
!pip install -U RelevanceAI[notebook]==2.0.0
# remove `!` if running the line in a terminal
!pip install vectorhub[encoders-text-tfhub]



#Set up Client

To use Relevance AI, a client object must be instantiated. This needs an API_key and a project name. These can be generated/access directly at https://cloud.relevance.ai/ or simply by running the cell below and following the link and the guide:


In [ ]:

from relevanceai import Client

"""
You can sign up/login and find your credentials here: https://cloud.relevance.ai/sdk/api
Once you have signed up, click on the value under `Activation token` and paste it here
"""
client = Client()




# Vector Search

## 1) Data

For this quickstart we will be using a sample e-commerce dataset. Alternatively, you can use your own dataset for the different steps.

In [ ]:
import pandas as pd
from relevanceai.utils.datasets import get_ecommerce_dataset_clean

# Retrieve our sample dataset. - This comes in the form of a list of documents.
documents = get_ecommerce_dataset_clean()

pd.DataFrame.from_dict(documents).head()


## 2) Encode

In [ ]:
from vectorhub.encoders.text.tfhub import USE2Vec
model = USE2Vec()

documents = model.encode_documents(fields=['product_title'], documents=documents)



In [ ]:
documents[0].keys()

We can see that there is now a field called `product_title_use_vector_` in our data. 
`_use_vector_` is the name assigned to the model in Vectorhub and we use it when generating vectors. 
If you prefer a different name, simply modify the `__name__` attribute via
`enc.__name__ = "model_name_goes_here"`

## 3) Insert

Uploading our documents into a dataset called `quickstart-example`.
Note that each document should have a field called '_id'. Here, we generate a unique identifier using the Python uuid package.

In [ ]:
import uuid

for d in documents:
  d['_id'] = uuid.uuid4().__str__()    # Each document must have a field '_id'

ds = client.Dataset("quickstart_text_searc")
ds.insert_documents(documents)


## 4) Search

Note that our dataset includes vectors generated by Universal Sentence Encoder. Therefore, in this step, we first vectorize the query using the same encoder to be able to search among the similarly generated vectors.

In [ ]:
query = 'Gift for my son'
query_vector = model.encode(query)


In [ ]:
multivector_query=[
    { "vector": query_vector, "fields": ["product_title_use_vector_"]}
]
results = ds.vector_search(
    multivector_query=multivector_query,
    page_size=5
)


You can now visit the dashboard at https://cloud.relevance.ai/sdk/search


In [ ]:
from relevanceai import show_json

print('=== QUERY === ')
print(query)

print('=== RESULTS ===')
show_json(results, image_fields=["product_image"], text_fields=["product_title"])




,product_title,_id
0,Animale Men's 3-piece Gift Set,92f64d5c-a10b-4bf2-9dc9-eda6e24e771e
1,Children's DC Shoes Spartan Hi Teal/Black,f4152203-4050-49c0-b759-26ce82329426
2,New Gift Unisex's Men Silver Stainless Steel Cross Pendant Necklace Chain gold,2f233c49-27cd-4f5a-bafe-2f9dd1a5af35
3,Crocs Boy (Toddler) 'Classic Kids' Synthetic Casual Shoes,96e26654-da0d-4ada-b800-2a4c0e3438dd
4,PS4 - Playstation 4 Console,4630632a-ceba-4428-b575-229393876dbb
